In [2]:
from langchain_openai import AzureChatOpenAI, ChatOpenAI
import os
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
# os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
# os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"


os.environ["AZURE_OPENAI_API_KEY"] = "aa183bb914bb4858b15bed161fb47ba5"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://bxcl-prod.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o-mini"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=1,
    n = 3,
    max_retries=5, request_timeout=600
)

openai_model = ChatOpenAI(model = 'gpt-3.5-turbo', base_url="https://api.chatanywhere.tech/v1", api_key="sk-bLZSHx4pKfPRZkYyIyyvUHSEjrlqj5sh2QIsxOM23yJnyoGD")
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
prompt = """Q: What state is home to the university that is represented in sports by George Washington Colonials
men’s basketball?
A: Washington, D.C.
Q: Who lists Pramatha Chaudhuri as an influence and wrote Jana Gana Mana?
A: Bharoto Bhagyo Bidhata.
Q: Who was the artist nominated for an award for You Drive Me Crazy?
A: Jason Allen Alexander.
Q: What person born in Siegen influenced the work of Vincent Van Gogh?
A: Peter Paul Rubens.
Q: What is the country close to Russia where Mikheil Saakashvii holds a government position?
A: Georgia.
A: Heroin.
Q: What drug did the actor who portrayed the character Urethane Wheels Guy overdosed on?
Q: {Query}
A: """

In [5]:
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_io_prompt = FewShotPromptTemplate(
        examples=[
        {
            "query": "who was real housewives brandi married to",
            "output": "Eddie Cibrian",        }, 
        {
            "query": "who did ben stiller play in megamind",
            "output": "Bernard",
        }, 
        {
            "query": "who was real housewives brandi married to",
            "output": "Eddie Cibrian",
        },
        {
            "query": "what time zone am i in cleveland ohio",
            "output": "Eastern Time Zone",
        }],
        example_prompt=PromptTemplate.from_template("""
Query: {query}
Output: {output}"""),
        prefix=
        """Based on the given query, answer the question. Directly output the answer without explanation""",
        suffix=
        "Query: {query}\n"
        "Output:",
        input_variables=["query"],
)




In [ ]:
graph_intepretable_prompt = PromptTemplate(input_variables=["Query"], template=
prompt)
io_chain = LLMChain(llm=model, prompt=few_shot_io_prompt, verbose=False)
io_chain_open = LLMChain(llm=openai_model, prompt=few_shot_io_prompt, verbose=False)
content = io_chain.invoke('what is the name of justin bieber brother')

In [7]:
import json
with open('./data/merged/WebQSP_test.json', 'r',encoding='utf-8') as f:
    test_data = json.load(f)
print(len(test_data))

1639


In [11]:
from typing import List
import random
def save_to_json(data: List, data_path=''):
    if not os.path.isfile(data_path):
        # 文件不存在，创建新列表并写入文件
        with open(data_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        return
    try:
        # 尝试读取现有文件
        with open(data_path, 'r', encoding='utf-8') as file:
            # 加载现有的JSON数据
            existing_data = json.load(file)
            existing_data.extend(data)
    except json.JSONDecodeError:
        # 文件不是有效的JSON，打印错误信息并退出
        print(f"文件 {data_path} 不是有效的JSON格式。")
        return
    except ValueError as e:
        # 打印错误信息并退出
        print(e)
        return
    # 将更新后的数据写回文件
    with open(data_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

In [ ]:
outputs = []
for index in range(1350, len(test_data)):
    data_input = test_data[index]['question']
    try:
        output = io_chain.invoke({'query': data_input})['text']
    except:
        output = io_chain_open.invoke({'query': data_input})['text']
    outputs.append({'qid': test_data[index]['ID'], 'answer': output})
    if index % 10 == 0:
        print(f'Saving {index}')
        save_to_json(outputs, './output/ablation/webqsp_io.json')
        outputs = []

In [24]:
with open('./output/ablation/webqsp_io.json', 'r') as f:
    webqsp = json.load(f)
print(len(webqsp))


1639


In [27]:
from src.sparql_utils import *

In [31]:
count = 0
for index in range(len(webqsp)):
    hit = 0
    assert webqsp[index]['qid'] == test_data[index]['ID']
    labels = [get_label(ans) if ans.startswith('m.') else ans for ans in test_data[index]['answer']]
    for label in labels:
        if label and label in webqsp[index]['answer']:
            hit = 1
    if hit == 1:
        count += 1
    else:
        break

In [29]:
count / len(webqsp)

0.46918852959121415